In [1]:
pip install fdtd


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import fdtd
import numpy as np


In [ ]:
fdtd.set_backend("torch")

In [ ]:
grid=fdtd.Grid(
    shape=(100,100,1),
    grid_spacing= 1,
    permittivity= 1.0,
    permeability= 1.0,
    #courant_number: float = None,
)

In [ ]:
Eps=np.ones((100,100))
for i in range(10):
    for j in range(10):
        Eps[i+20][j+30]=1.5

In [ ]:
for i in range(100):
    for j in range(100):
        grid[i, j, 0] = fdtd.Object(permittivity=Eps[i][j]**2)

In [ ]:
grid[i, j, 0] = fdtd.Object(permittivity=1.5**2,name="object2"+)

In [ ]:
grid[49:50, 49:51, 0]=fdtd.LineSource(period=1/3e8,name="source2")


In [ ]:
grid[50, 50, 0]=fdtd.PointSource( name="object4")

In [ ]:
grid[8e-6, :, 0] = fdtd.LineDetector(name="detector")

In [ ]:
# grid[50:52, 60:70, 0] = fdtd.LineSource(
#      period = 1/3e8,cycle=1,pulse=True, name="source"
# )

In [ ]:
# x boundaries
grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")

# y boundaries
grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")

In [ ]:
grid.run(total_time=100)

In [ ]:
grid.visualize(z=0)

In [ ]:
grid.save_simulation("sim1")

In [ ]:
grid.save_data()

In [ ]:
grid.

In [ ]:
grid.update_H()
print(grid)

In [ ]:
def curl_E(E):
    curl_E = np.zeros(E.shape)
    curl_E[:,:-1,:,0] += E[:,1:,:,2] - E[:,:-1,:,2]
    curl_E[:,:,:-1,0] -= E[:,:,1:,1] - E[:,:,:-1,1]

    curl_E[:,:,:-1,1] += E[:,:,1:,0] - E[:,:,:-1,0]
    curl_E[:-1,:,:,1] -= E[1:,:,:,2] - E[:-1,:,:,2]

    curl_E[:-1,:,:,2] += E[1:,:,:,1] - E[:-1,:,:,1]
    curl_E[:,:-1,:,2] -= E[:,1:,:,0] - E[:,:-1,:,0]
    return curl_E

In [ ]:
def curl_H(H):
    curl_H = np.zeros(H.shape)

    curl_H[:,1:,:,0] += H[:,1:,:,2] - H[:,:-1,:,2]
    curl_H[:,:,1:,0] -= H[:,:,1:,1] - H[:,:,:-1,1]

    curl_H[:,:,1:,1] += H[:,:,1:,0] - H[:,:,:-1,0]
    curl_H[1:,:,:,1] -= H[1:,:,:,2] - H[:-1,:,:,2]

    curl_H[1:,:,:,2] += H[1:,:,:,1] - H[:-1,:,:,1]
    curl_H[:,1:,:,2] -= H[:,1:,:,0] - H[:,:-1,:,0]
    return curl_H

In [ ]:
    E  += (c*dt/du)*inv(ε)*curl_H
    H  -= (c*dt/du)*inv(µ)*curl_E

In [ ]:
class Grid:
    # ... [initialization]

    def step(self):
        self.update_E()
        self.update_H()

    def update_E(self):
        self.E += self.courant_number * self.inverse_permittivity * curl_H(self.H)

    def update_H(self):
        self.H -= self.courant_number * self.inverse_permeability * curl_E(self.E)